In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [6]:
train = pd.read_csv('train.dat')
test = pd.read_csv('test.dat')
train

,# x1,x2,x3,y
0,2.7,0.7,-0.4,5.5
1,3.6,0.3,0.4,7.0
2,3.0,0.7,-0.2,6.0
3,2.7,1.0,-0.6,5.6
4,2.1,0.2,-1.0,4.2
...,...,...,...,...
95,0.9,0.5,-0.6,2.1
96,2.9,0.2,-0.8,5.4
97,0.1,0.3,0.7,0.2
98,4.1,0.1,0.9,8.0


In [3]:
test

,# x1,x2,x3,y
0,4.5,0.4,-0.4,9.5
1,3.9,0.9,-0.3,7.8
2,1.7,0.1,0.0,3.4
3,0.4,0.9,0.5,0.5
4,2.0,0.9,-0.3,4.0
...,...,...,...,...
95,4.8,0.3,1.0,9.7
96,1.8,0.5,-0.3,3.6
97,1.8,0.4,-0.1,3.3
98,0.1,0.2,-0.2,0.1
